# Find My New City

### Many people wish to move cities and they are unsure on which city they should move to, one natural option to consider is moving to a city that is the most similar to your own. This project will solve that problem, it will take a users city based in the US as an input (such as a university student who has received multiple offers in different cities), after this then the program will cluster and classify cities based on their similarity to each other. After this is complete the city/cities which are most similar to the users original city will be displayed for them as well as the cities level of similarity to the original.

### To complete the project I will be using US cities data and the FourSquare API together to obtain information about the various cities. Mainly data such as venues and venue types that can be found in the city. This data will be used as dimensions to classify the cities into different clusters.